# Start My Batfish Journey

![image](./batfish_snapshot/forwarding-change-validation/Pic.png)

In [1]:
# Import packages
%run startup.py
load_questions()
bf_session.host = 'localhost'


## Initializing a new snapshot

In [2]:
NETWORK_NAME = 'forwarding_change_validation'
SNAPSHOT_NAME = 'base'
SNAPSHOT_DIR = './batfish_snapshot/forwarding-change-validation/base'


bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_DIR, name=SNAPSHOT_NAME, overwrite=True)


Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


'base'

## Get status of parsed files

In [3]:
parse_status = bfq.fileParseStatus().answer().frame()   #answer is json, frame is pandas dataframe
parse_status#[parse_status['Status'] != 'PASSED']

,File_Name,Status,Nodes
0,configs/border1.cfg,PARTIALLY_UNRECOGNIZED,['border1']
1,configs/border2.cfg,PASSED,['border2']
2,configs/core1.cfg,PASSED,['core1']
3,configs/core2.cfg,PASSED,['core2']
4,configs/leaf1.cfg,PASSED,['leaf1']
5,configs/spine1.cfg,PASSED,['spine1']
6,configs/spine2.cfg,PASSED,['spine2']
7,hosts/host-db.json,PASSED,['host-db']
8,hosts/host-www.json,PASSED,['host-www']


In [4]:
load_questions()  #load all questions
bfq.initIssues().answer().frame()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,None,[configs/border1.cfg:[18]],Parse warning,This syntax is unrecognized,mmi polling-interval 60,[cisco_configuration]
1,None,[configs/border1.cfg:[19]],Parse warning,This syntax is unrecognized,no mmi auto-configure,[cisco_configuration]
2,None,[configs/border1.cfg:[20]],Parse warning,This syntax is unrecognized,no mmi pvc,[stanza cisco_configuration]
3,None,[configs/border1.cfg:[21]],Parse warning,This syntax is unrecognized,mmi snmp-timeout 180,[cisco_configuration]


# Batfish questins
https://www.batfish.org/assets/cheat-sheet.pdf


## Node properties

In [5]:
# Extract the properties of all nodes whose names contain 'border'
node_properties = bfq.nodeProperties(nodes="/border/").answer().frame()

In [6]:
# View what columns (properties) are present in the answer
node_properties.columns

Index(['Node', 'AS_Path_Access_Lists', 'Authentication_Key_Chains',
       'Community_Lists', 'Configuration_Format', 'DNS_Servers',
       'DNS_Source_Interface', 'Default_Cross_Zone_Action',
       'Default_Inbound_Action', 'Domain_Name', 'Hostname', 'IKE_Phase1_Keys',
       'IKE_Phase1_Policies', 'IKE_Phase1_Proposals', 'IP6_Access_Lists',
       'IP_Access_Lists', 'IPsec_Peer_Configs', 'IPsec_Phase2_Policies',
       'IPsec_Phase2_Proposals', 'Interfaces', 'Logging_Servers',
       'Logging_Source_Interface', 'NTP_Servers', 'NTP_Source_Interface',
       'PBR_Policies', 'Route6_Filter_Lists', 'Route_Filter_Lists',
       'Routing_Policies', 'SNMP_Source_Interface', 'SNMP_Trap_Servers',
       'TACACS_Servers', 'TACACS_Source_Interface', 'VRFs', 'Zones'],
      dtype='object')

In [7]:
# To extract only a subset of properties, use the properties parameter
node_properties_trunc = bfq.nodeProperties(nodes="/border/", properties="Domain_Name,NTP_Servers,Interfaces").answer().frame()

node_properties_trunc

,Node,Domain_Name,Interfaces,NTP_Servers
0,border1,lab.local,"['Ethernet0/0', 'GigabitEthernet0/0', 'GigabitEthernet1/0', 'GigabitEthernet2/0', 'Loopback0']","['18.18.18.18', '23.23.23.23']"
1,border2,lab.local,"['Ethernet0/0', 'GigabitEthernet0/0', 'GigabitEthernet1/0', 'GigabitEthernet2/0', 'Loopback0']",['18.18.18.18']


## Interface properties

In [8]:
interface_properties = bfq.interfaceProperties(nodes="/border/", properties="Bandwidth,VRF,Primary_Address").answer().frame()

In [9]:
interface_properties[interface_properties['Primary_Address'].str.match("10.12", na=False)]

,Interface,Bandwidth,Primary_Address,VRF
8,border1[GigabitEthernet0/0],1e+09,10.12.11.2/24,default


## Network adjacencies

In [10]:
edge = bfq.edges(edgeType="bgp").answer().frame()
edge

,Node,IP,Interface,AS_Number,Remote_Node,Remote_IP,Remote_Interface,Remote_AS_Number
0,border2,2.1.1.2,None,2,core2,2.1.2.2,None,2
1,leaf1,2.34.201.4,None,65001,spine2,2.34.201.3,None,2
2,core1,2.1.2.1,None,2,border2,2.1.1.2,None,2
3,core1,2.1.2.1,None,2,border1,2.1.1.1,None,2
4,core1,2.1.2.1,None,2,spine2,2.1.3.2,None,2
5,border1,2.1.1.1,None,2,core2,2.1.2.2,None,2
6,spine2,2.1.3.2,None,2,core2,2.1.2.2,None,2
7,spine1,2.34.101.3,None,2,leaf1,2.34.101.4,None,65001
8,spine2,2.34.201.3,None,2,leaf1,2.34.201.4,None,65001
9,core2,2.1.2.2,None,2,spine1,2.1.3.1,None,2


## Routing

In [11]:
routes_df = bfq.routes().answer().frame()
routes_df[routes_df['Node']=='border1'][routes_df['Protocol']=='connected']

<ipython-input-11-465fb7391ba6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  routes_df[routes_df['Node']=='border1'][routes_df['Protocol']=='connected']


,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
19,border1,default,2.1.1.1/32,None,AUTO/NONE(-1l),Loopback0,connected,0,0,None
50,border1,default,2.12.11.0/24,None,AUTO/NONE(-1l),GigabitEthernet1/0,connected,0,0,None
127,border1,default,10.12.11.0/24,None,AUTO/NONE(-1l),GigabitEthernet0/0,connected,0,0,None
132,border1,default,2.12.12.0/24,None,AUTO/NONE(-1l),GigabitEthernet2/0,connected,0,0,None


## Forward change 

In [12]:
answer = bfq.traceroute(
    startLocation="@enter(/border/[GigabitEthernet0/0])",
    headers=HeaderConstraints(dstIps="/host/")
).answer()
show(answer.frame())

,Flow,Traces,TraceCount
0,Start Location: border1 interface=GigabitEthernet0/0Src IP: 10.12.11.1Src Port: 49152Dst IP: 2.128.0.1Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: border1 RECEIVED(GigabitEthernet0/0) PERMITTED(OUTSIDE_TO_INSIDE (INGRESS_FILTER)) FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet1/0)2. node: core1 RECEIVED(GigabitEthernet0/0) FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet3/0)3. node: spine2 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/30, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(RESTRICT_HOST_TRAFFIC_OUT (EGRESS_FILTER)) TRANSMITTED(GigabitEthernet2/0)5. node: host-db RECEIVED(eth0) ACCEPTED(eth0)",1
1,Start Location: border2 interface=GigabitEthernet0/0Src IP: 10.23.21.1Src Port: 49152Dst IP: 2.128.0.1Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: border2 RECEIVED(GigabitEthernet0/0) PERMITTED(OUTSIDE_TO_INSIDE (INGRESS_FILTER)) FORWARDED(ARP IP: 2.12.21.2, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)2. node: core1 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet3/0)3. node: spine2 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/30, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(RESTRICT_HOST_TRAFFIC_OUT (EGRESS_FILTER)) TRANSMITTED(GigabitEthernet2/0)5. node: host-db RECEIVED(eth0) ACCEPTED(eth0)",1


In [13]:
# Search for any traffic routed through core1
answer = bfq.reachability(
    pathConstraints=PathConstraints(
        startLocation="@enter(/border/[GigabitEthernet0/0])",
        transitLocations="core1"),
    headers=HeaderConstraints(dstIps="/host-www/"),
    actions="SUCCESS,FAILURE"
).answer().frame()
show(answer)

,Flow,Traces,TraceCount
0,Start Location: border1 interface=GigabitEthernet0/0Src IP: 10.12.11.1Src Port: 49152Dst IP: 2.128.1.1Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: border1 RECEIVED(GigabitEthernet0/0) PERMITTED(OUTSIDE_TO_INSIDE (INGRESS_FILTER)) FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet1/0)2. node: core1 RECEIVED(GigabitEthernet0/0) FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet3/0)3. node: spine2 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet3/0, Routes: [connected (Network: 2.128.1.0/30, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(RESTRICT_HOST_TRAFFIC_OUT (EGRESS_FILTER)) TRANSMITTED(GigabitEthernet3/0)5. node: host-www RECEIVED(eth0) ACCEPTED(eth0)",1
1,Start Location: border2 interface=GigabitEthernet0/0Src IP: 10.23.21.1Src Port: 49152Dst IP: 2.128.1.1Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: border2 RECEIVED(GigabitEthernet0/0) PERMITTED(OUTSIDE_TO_INSIDE (INGRESS_FILTER)) FORWARDED(ARP IP: 2.12.21.2, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)2. node: core1 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet3/0)3. node: spine2 RECEIVED(GigabitEthernet1/0) FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.1.0/30, Next Hop IP:2.34.201.4)]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet3/0, Routes: [connected (Network: 2.128.1.0/30, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(RESTRICT_HOST_TRAFFIC_OUT (EGRESS_FILTER)) TRANSMITTED(GigabitEthernet3/0)5. node: host-www RECEIVED(eth0) ACCEPTED(eth0)",1
